# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 30 2022 9:31AM,247628,10,SOTAH0000795,"Nextsource Biotechnology, LLC",Released
1,Sep 30 2022 9:31AM,247628,10,SOTAH0000796,"Nextsource Biotechnology, LLC",Released
2,Sep 30 2022 9:28AM,247627,12,HH8048,Hush Hush,Executing
3,Sep 30 2022 9:27AM,247626,12,HH-37199,Hush Hush,Executing
4,Sep 30 2022 9:27AM,247626,12,HH-37200,Hush Hush,Executing
5,Sep 30 2022 9:27AM,247626,12,HH-37201,Hush Hush,Executing
6,Sep 30 2022 9:27AM,247626,12,HH-37202,Hush Hush,Executing
7,Sep 30 2022 9:27AM,247626,12,HH-37204,Hush Hush,Executing
8,Sep 30 2022 9:27AM,247626,12,HH-37205,Hush Hush,Executing
9,Sep 30 2022 9:27AM,247626,12,HH-37206,Hush Hush,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,247625,Executing,1
25,247626,Executing,7
26,247626,Released,10
27,247627,Executing,1
28,247628,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247624,NaN,1.0,NaN
247625,NaN,1.0,NaN
247626,NaN,7.0,10.0
247627,NaN,1.0,NaN
247628,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0.0,0.0,1.0
247567,0.0,0.0,18.0
247576,0.0,0.0,31.0
247578,9.0,0.0,2.0
247595,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0,0,1
247567,0,0,18
247576,0,0,31
247578,9,0,2
247595,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,0,0,1
1,247567,0,0,18
2,247576,0,0,31
3,247578,9,0,2
4,247595,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,,,1
1,247567,,,18
2,247576,,,31
3,247578,9,,2
4,247595,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 30 2022 9:31AM,247628,10,"Nextsource Biotechnology, LLC"
2,Sep 30 2022 9:28AM,247627,12,Hush Hush
3,Sep 30 2022 9:27AM,247626,12,Hush Hush
20,Sep 30 2022 9:25AM,247625,19,"GCH Granules USA, Inc."
21,Sep 30 2022 9:17AM,247621,20,"ACI Healthcare USA, Inc."
22,Sep 30 2022 9:17AM,247623,10,ISDIN Corporation
53,Sep 30 2022 9:15AM,247624,22,"NBTY Global, Inc."
54,Sep 30 2022 9:14AM,247622,22,"NBTY Global, Inc."
55,Sep 30 2022 9:07AM,247620,19,ACG North America LLC
57,Sep 30 2022 9:05AM,247619,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 30 2022 9:31AM,247628,10,"Nextsource Biotechnology, LLC",,,2
1,Sep 30 2022 9:28AM,247627,12,Hush Hush,,1,
2,Sep 30 2022 9:27AM,247626,12,Hush Hush,,7,10
3,Sep 30 2022 9:25AM,247625,19,"GCH Granules USA, Inc.",,1,
4,Sep 30 2022 9:17AM,247621,20,"ACI Healthcare USA, Inc.",,1,
5,Sep 30 2022 9:17AM,247623,10,ISDIN Corporation,,,31
6,Sep 30 2022 9:15AM,247624,22,"NBTY Global, Inc.",,1,
7,Sep 30 2022 9:14AM,247622,22,"NBTY Global, Inc.",,,1
8,Sep 30 2022 9:07AM,247620,19,ACG North America LLC,,,2
9,Sep 30 2022 9:05AM,247619,10,ISDIN Corporation,,,46


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 9:31AM,247628,10,"Nextsource Biotechnology, LLC",2,,
1,Sep 30 2022 9:28AM,247627,12,Hush Hush,,1,
2,Sep 30 2022 9:27AM,247626,12,Hush Hush,10,7,
3,Sep 30 2022 9:25AM,247625,19,"GCH Granules USA, Inc.",,1,
4,Sep 30 2022 9:17AM,247621,20,"ACI Healthcare USA, Inc.",,1,
5,Sep 30 2022 9:17AM,247623,10,ISDIN Corporation,31,,
6,Sep 30 2022 9:15AM,247624,22,"NBTY Global, Inc.",,1,
7,Sep 30 2022 9:14AM,247622,22,"NBTY Global, Inc.",1,,
8,Sep 30 2022 9:07AM,247620,19,ACG North America LLC,2,,
9,Sep 30 2022 9:05AM,247619,10,ISDIN Corporation,46,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 9:31AM,247628,10,"Nextsource Biotechnology, LLC",2,,
1,Sep 30 2022 9:28AM,247627,12,Hush Hush,,1,
2,Sep 30 2022 9:27AM,247626,12,Hush Hush,10,7,
3,Sep 30 2022 9:25AM,247625,19,"GCH Granules USA, Inc.",,1,
4,Sep 30 2022 9:17AM,247621,20,"ACI Healthcare USA, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 9:31AM,247628,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
1,Sep 30 2022 9:28AM,247627,12,Hush Hush,NaN,1.0,NaN
2,Sep 30 2022 9:27AM,247626,12,Hush Hush,10.0,7.0,NaN
3,Sep 30 2022 9:25AM,247625,19,"GCH Granules USA, Inc.",NaN,1.0,NaN
4,Sep 30 2022 9:17AM,247621,20,"ACI Healthcare USA, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 9:31AM,247628,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
1,Sep 30 2022 9:28AM,247627,12,Hush Hush,0.0,1.0,0.0
2,Sep 30 2022 9:27AM,247626,12,Hush Hush,10.0,7.0,0.0
3,Sep 30 2022 9:25AM,247625,19,"GCH Granules USA, Inc.",0.0,1.0,0.0
4,Sep 30 2022 9:17AM,247621,20,"ACI Healthcare USA, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2228450,134.0,29.0,9.0
12,495253,10.0,8.0,0.0
15,495179,33.0,0.0,0.0
19,1238077,4.0,20.0,0.0
20,247621,0.0,1.0,0.0
21,990214,1.0,3.0,0.0
22,742841,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2228450,134.0,29.0,9.0
1,12,495253,10.0,8.0,0.0
2,15,495179,33.0,0.0,0.0
3,19,1238077,4.0,20.0,0.0
4,20,247621,0.0,1.0,0.0
5,21,990214,1.0,3.0,0.0
6,22,742841,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,134.0,29.0,9.0
1,12,10.0,8.0,0.0
2,15,33.0,0.0,0.0
3,19,4.0,20.0,0.0
4,20,0.0,1.0,0.0
5,21,1.0,3.0,0.0
6,22,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,134.0
1,12,Released,10.0
2,15,Released,33.0
3,19,Released,4.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,29.0,8.0,0.0,20.0,1.0,3.0,1.0
Released,134.0,10.0,33.0,4.0,0.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,29.0,8.0,0.0,20.0,1.0,3.0,1.0
2,Released,134.0,10.0,33.0,4.0,0.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,29.0,8.0,0.0,20.0,1.0,3.0,1.0
2,Released,134.0,10.0,33.0,4.0,0.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()